In [1]:
import pandas as pd
print("Codespaces fonctionne !") #test

Codespaces fonctionne !


In [2]:
import os
os.chdir('/workspaces/projet-iadata')  # Change le répertoire vers la racine
print(os.getcwd()) 


/workspaces/projet-iadata


## NETTOYAGE DE DONNEES

### VERIFIER LES SEPARATEURS

In [5]:
#lecture de fichier
file_path='./data/brut/MENSQ_01_latest-2023-2024.csv'
print(file_path)

./data/brut/MENSQ_01_latest-2023-2024.csv


In [ ]:
#ce fichier ne peut etre ouvert que apres l'avoir formaté
mensq=pd.read_csv(file_path, sep=';')


print(mensq.columns)#verif des colonnes


Index(['NUM_POSTE', 'NOM_USUEL', 'LAT', 'LON', 'ALTI', 'AAAAMM', 'RR', 'QRR',
       'NBRR', 'RR_ME',
       ...
       'QNEIGETOTM', 'NEIGETOTAB', 'QNEIGETOTAB', 'NEIGETOTABDAT',
       'NBJNEIGETOT1', 'NBJNEIGETOT10', 'NBJNEIGETOT30', 'NBJGREL', 'NBJORAG',
       'NBJBROU'],
      dtype='object', length=162)


##### RENOMMER LES COLONNNES

In [ ]:
#
# Renommerpour simplifier l' usage
mensq.rename(columns={
    '﻿Nom du festival': 'NomFestival',
   
}, inplace=True)

# verification des colonnes renommées
print(mensq.columns)



Index(['NomFestival', 'Envergure', 'Region', 'Departement', 'Commune',
       'CodePostal', 'CodeInseeCommune', 'CodeInseeEPCI', 'LibelleEPCI',
       'NumVoie', 'TypeVoie', 'NomVoie', 'AdressePostale', 'ComplementAdresse',
       'SiteInternet', 'Email', 'DecennieCreation', 'AnneeCreation',
       'Discipline', 'CategorieSpectacleVivant', 'CategorieMusique',
       'CategorieMusiqueCNM', 'CategorieCinema', 'CategorieArtsVisuels',
       'CategorieLitterature', 'PeriodeFestival', 'IdAgenceA', 'Identifiant',
       'GeocodageXY', 'IdCNM'],
      dtype='object')


### CALCUL POURCENTAGE DE VALEURS NULLES

In [ ]:
#pourcentage de valeurs nulles
missing_percent= mensq.isnull().mean() * 100

print(missing_percent)

NomFestival                  0.000000
Envergure                   73.925580
Region                       0.000000
Departement                  0.356996
Commune                      0.356996
CodePostal                   0.233420
CodeInseeCommune             0.013731
CodeInseeEPCI                0.494302
LibelleEPCI                  0.494302
NumVoie                     69.724015
TypeVoie                    60.565701
NomVoie                     51.929150
AdressePostale              51.778113
ComplementAdresse           69.765207
SiteInternet                 9.954689
Email                       23.396952
DecennieCreation             5.066593
AnneeCreation               19.881917
Discipline                   0.000000
CategorieSpectacleVivant    80.021969
CategorieMusique            64.039544
CategorieMusiqueCNM         86.846080
CategorieCinema             93.162158
CategorieArtsVisuels        96.155430
CategorieLitterature        94.329260
PeriodeFestival              0.823836
IdAgenceA   

### CATEGORISER LES COL SELON LES % VALEURS MANQ...

#### colonnes à supprimer ( x% >85 )

In [7]:
high_missing = missing_percent[missing_percent>85].index

#### Remplir par  "Inconnu"  (5 < x% <= 50)

In [8]:
medium_missing= missing_percent[(missing_percent>5) & (missing_percent<=50)].index

#### Supprimer les lignes concernées  (x% <= 5)

In [9]:
low_missing = missing_percent[missing_percent <= 5].index  

#### Nettoyage des données
#### Supprimer les colonnes avec plus de 85% de valeurs manquantes

In [ ]:
mensq.drop(columns=high_missing, inplace=True)
print(f"Colonnes supprimées : {list(high_missing)}")

Colonnes supprimées : ['CategorieMusiqueCNM', 'CategorieCinema', 'CategorieArtsVisuels', 'CategorieLitterature', 'IdAgenceA', 'IdCNM']


#### Remplir les colonnes avec entre 5% et 50% de valeurs manquantes

In [11]:
for col in medium_missing:
    festivals[col].fillna("Inconnu", inplace=True)
print(f"Colonnes remplies avec 'Inconnu' : {list(medium_missing)}")

Colonnes remplies avec 'Inconnu' : ['SiteInternet', 'Email', 'DecennieCreation', 'AnneeCreation']


/tmp/ipykernel_2265/2740360676.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  festivals[col].fillna("Inconnu", inplace=True)


####  Supprimer les lignes pour les colonnes avec moins de 5% de valeurs manquantes

In [12]:
festivals.dropna(subset=low_missing, inplace=True)
print(f"Colonnes où les lignes avec valeurs manquantes ont été supprimées : {list(low_missing)}")

Colonnes où les lignes avec valeurs manquantes ont été supprimées : ['NomFestival', 'Region', 'Departement', 'Commune', 'CodePostal', 'CodeInseeCommune', 'CodeInseeEPCI', 'LibelleEPCI', 'Discipline', 'PeriodeFestival', 'Identifiant', 'GeocodageXY']


#### Afficher les données nettoyées

In [13]:
print("appercu des données nettoyées")
print(festivals.head())
print(f"Nombre de lignes paprès le nettoyage : {len(festivals)}")

appercu des données nettoyées
                                  NomFestival  Envergure  \
0                        Les Nuits (Re)Belles        NaN   
1                               Y'a pas l'feu  Régionale   
2                 Festival du jeune art russe        NaN   
3      Festival de musique de Cordes-sur-Ciel        NaN   
4  Les rencontres de la cinémathèque montagne        NaN   

                       Region      Departement          Commune CodePostal  \
0     Bourgogne-Franche-Comté             Jura         Brainans      39800   
1             Hauts-de-France    Pas-de-Calais          Cambrin      62149   
2  Provence-Alpes-Côte d'Azur  Alpes-Maritimes           Cannes       6414   
3                   Occitanie             Tarn  Cordes-sur-Ciel      81170   
4  Provence-Alpes-Côte d'Azur     Hautes-Alpes              Gap       5000   

  CodeInseeCommune CodeInseeEPCI                               LibelleEPCI  \
0            39073     200071595  CC Arbois, Poligny, Salins, 

In [14]:
#### FILTRER LES COLONNES NECESSAIRES

colonnes_utiles=[
     'NomFestival', 'Envergure', 'Region', 'Departement', 'Commune',
    'CodePostal', 'Discipline', 'PeriodeFestival', 'GeocodageXY'
]

## GARDER LES COLONNES UTILES POUR L'ANALYSE
festivals_reduits=festivals[colonnes_utiles]

In [15]:
##APPERCU DES DONNEES REDUITES
print("colonnes finales concervées")
print(festivals_reduits.columns)
print(festivals_reduits.head())

colonnes finales concervées
Index(['NomFestival', 'Envergure', 'Region', 'Departement', 'Commune',
       'CodePostal', 'Discipline', 'PeriodeFestival', 'GeocodageXY'],
      dtype='object')
                                  NomFestival  Envergure  \
0                        Les Nuits (Re)Belles        NaN   
1                               Y'a pas l'feu  Régionale   
2                 Festival du jeune art russe        NaN   
3      Festival de musique de Cordes-sur-Ciel        NaN   
4  Les rencontres de la cinémathèque montagne        NaN   

                       Region      Departement          Commune CodePostal  \
0     Bourgogne-Franche-Comté             Jura         Brainans      39800   
1             Hauts-de-France    Pas-de-Calais          Cambrin      62149   
2  Provence-Alpes-Côte d'Azur  Alpes-Maritimes           Cannes       6414   
3                   Occitanie             Tarn  Cordes-sur-Ciel      81170   
4  Provence-Alpes-Côte d'Azur     Hautes-Alpes            

#### SAUVEGARDE DES DONNEES DANS UN NOUVEAU FICHIER

In [18]:
# Vérification des modifications
print(festivals_reduits.describe())

festivals_reduits.to_csv('./data/prepared/festi_clean.csv', index=False) 

           NomFestival       Envergure                Region  \
count             7184            1866                  7184   
unique            7079              70                    18   
top     Salon du livre  Intercommunale  Auvergne-Rhône-Alpes   
freq                 6             764                   945   

             Departement Commune CodePostal Discipline  \
count               7184    7184       7184       7184   
unique               108    3134       2686          6   
top     Bouches-du-Rhône   Paris      35000    Musique   
freq                 300     295         70       3183   

                       PeriodeFestival      GeocodageXY  
count                             7184             7184  
unique                              22             3135  
top     Saison (21 juin - 5 septembre)  48.8567, 2.3508  
freq                              2627              123  
